In [1]:
%env AZURE_EXTENSION_DIR=/home/schrodinger/automl/sdk-cli-v2/src/cli/src
%env AZURE_ML_CLI_PRIVATE_FEATURES_ENABLED=true

env: AZURE_EXTENSION_DIR=/home/schrodinger/automl/sdk-cli-v2/src/cli/src
env: AZURE_ML_CLI_PRIVATE_FEATURES_ENABLED=true


# Setup

## Imports

In [14]:
import os

from azure.ml import MLClient
from azure.ml.entities import Endpoint, ManagedOnlineEndpoint, Environment, \
CodeConfiguration, ManagedOnlineDeployment, ManualScaleSettings, Code

import mlflow
from mlflow.tracking import MlflowClient

from azure.ml.entities._assets import Model

## Setting necessary context

In [4]:
subscription_id = '381b38e9-9840-4719-a5a0-61d9585e1e91'
resource_group_name = 'gasi_rg_neu'
workspace_name = "gasi_ws_neu"
experiment_name = "automl-classification-bmarketing-all"

## Initialize Azure ML Client

In [9]:
client = MLClient(subscription_id, resource_group_name, workspace_name=workspace_name)
assert client is not None

## Initialize MLFlow Client

The models and artifacts that are produced by AutoML can be accessed via. the MLFlow interface. Initialize the MLFlow client here, and set the backend as Azure ML, via. the MLFlow Client.

In [6]:
tracking_uri = "TODO --> Get this from MLClient"

################################################################################
# TODO: The API to get tracking URI is not yet available on Worksapce object.
from azureml.core import Workspace as WorkspaceV1
ws = WorkspaceV1(workspace_name=workspace_name, resource_group=resource_group_name, subscription_id=subscription_id)
tracking_uri = ws.get_mlflow_tracking_uri()
del ws
################################################################################

mlflow.set_tracking_uri(tracking_uri)
mlflow.set_experiment(experiment_name)

print("\nCurrent tracking uri: {}".format(mlflow.get_tracking_uri()))


Current tracking uri: azureml://northeurope.experiments.azureml.net/mlflow/v1.0/subscriptions/381b38e9-9840-4719-a5a0-61d9585e1e91/resourceGroups/gasi_rg_neu/providers/Microsoft.MachineLearningServices/workspaces/gasi_ws_neu?


# Retrieve the Best Trial

Or any trial based on a custom criteria. Use the MLFLowClient to access the results (such as Models, Artifacts, Metrics) of a previously completed AutoML Trial.

In [7]:
job_name = "AutoML_b120a44d-ecb4-4494-b644-f93c265e1028"

mlflow_client = MlflowClient()
mlflow_parent_run = mlflow_client.get_run(job_name)

best_child_run_id = mlflow_parent_run.data.tags["automl_best_child_run_id"]
print("Found best child run id: ", best_child_run_id)

best_run = mlflow_client.get_run(best_child_run_id)

Found best child run id:  AutoML_b120a44d-ecb4-4494-b644-f93c265e1028_0


# Download the best model locally

In [12]:
local_dir = "/tmp/artifact_downloads"
if not os.path.exists(local_dir):
    os.mkdir(local_dir)
local_path = mlflow_client.download_artifacts(best_run.info.run_id, "outputs", local_dir)
print("Artifacts downloaded in: {}".format(local_path))
print("Artifacts: {}".format(os.listdir(local_path)))

Artifacts downloaded in: /tmp/artifact_downloads/outputs
Artifacts: ['pipeline_graph.json', 'model_onnx.json', 'env_dependencies.json', 'scoring_file_v_1_0_0.py', 'model.pkl', 'conda.yaml', 'MLmodel', 'scoring_file_v_2_0_0.py', 'requirements.txt', 'conda_env_v_1_0_0.yml', 'model.onnx']


# Create Model on Azure ML

In [13]:
model_name = "AutoML_model"
azure_model = Model(name=model_name, version=1, local_path=os.path.join(local_path, "model.pkl"))
azure_model = client.models.create_or_update(azure_model)
azure_model

Uploading model.pkl: 100%|██████████| 396k/396k [00:00<00:00, 550kB/s]


Model({'is_anonymous': False, 'name': 'AutoML_model', 'description': None, 'tags': {}, 'properties': {}, 'id': '/subscriptions/381b38e9-9840-4719-a5a0-61d9585e1e91/resourceGroups/gasi_rg_neu/providers/Microsoft.MachineLearningServices/workspaces/gasi_ws_neu/models/AutoML_model/versions/1', 'base_path': './', 'creation_context': <azure.ml._restclient.v2021_03_01_preview.models._models_py3.SystemData object at 0x7f380993fbd0>, 'version': 1, 'datastore': '/subscriptions/381b38e9-9840-4719-a5a0-61d9585e1e91/resourceGroups/gasi_rg_neu/providers/Microsoft.MachineLearningServices/workspaces/gasi_ws_neu/datastores/workspaceblobstore', 'path': 'LocalUpload/381715692d299a7479fb8ed61cdc0b65/model.pkl', 'local_path': None, 'utc_time_created': None, 'flavors': {}})

# Create and deploy an endpoint

In [17]:
inference_script_file_name = os.path.join(local_path, "scoring_file_v_1_0_0.py")
conda_environment_yaml = os.path.join(local_path, "conda.yaml")

print("Inference File: ", inference_script_file_name)
print("Conda Environment File: ", conda_environment_yaml)

assert os.path.exists(inference_script_file_name)
assert os.path.exists(conda_environment_yaml)


# Prepare the deployment configuration
environment = Environment(
    name="environment-{}".format(best_run.info.run_id[:6]),
    version=1,
    path=".",
    conda_file=conda_environment_yaml,
    docker_image="mcr.microsoft.com/azureml/intelmpi2018.3-ubuntu16.04:20210301.v1",
)

code = Code(
    name="environment-{}".format(best_run.info.run_id[:6]),
    version=1,
    local_path=inference_script_file_name,
)
code_configuration = CodeConfiguration(
    code=code,
    scoring_script=inference_script_file_name
)

scale_settings = ManualScaleSettings(
    scale_type="Manual",
    min_instances=1,
    max_instances=2,
    instance_count=1
)
deployment = ManagedOnlineDeployment(
    name="deployment-{}".format(best_run.info.run_id[:6]),
    model=azure_model,
    environment=environment,
    code_configuration=code_configuration,
    instance_type="Standard_F2s_v2",
    scale_settings=scale_settings,
                                    )
online_endpoint = ManagedOnlineEndpoint(
    name="endpoint-{}".format(best_run.info.run_id[:6]),
    deployments=[deployment],
    description="Demo model deployment",
    tags={"deployed_using": "sdkv2"}
)
##### Loading from YAML
# endpoint = Endpoint.load("/home/schrodinger/automl/Easy-AutoML-MLOps/notebooks/3-automl-remote-compute-run/endpoint.yml")

try:
    client.endpoints.create(online_endpoint)
except Exception as e:
    import traceback
    print("Deployment failed: ", str(e))
    traceback.print_exc()

Inference File:  /tmp/artifact_downloads/outputs/scoring_file_v_1_0_0.py
Conda Environment File:  /tmp/artifact_downloads/outputs/conda.yaml


Uploading scoring_file_v_1_0_0.py: 100%|██████████| 2.91k/2.91k [00:00<00:00, 19.6kB/s]
The deployment request gasi_ws_neu-endpoint-automl-3240847 was accepted. ARM deployment URI for reference: 

https://ms.portal.azure.com/#blade/HubsExtension/DeploymentDetailsBlade/overview/id/%2Fsubscriptions%2F381b38e9-9840-4719-a5a0-61d9585e1e91%2FresourceGroups%2Fgasi_rg_neu%2Fproviders%2FMicrosoft.Resources%2Fdeployments%2Fgasi_ws_neu-endpoint-automl-3240847

Registering environment version (environment-AutoML:1)  Done (2s)

Registering model version (AutoML_model:1)  Done (2s)

Creating endpoint endpoint-automl 
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
  Done (2m 40s)

Creating deployment deployment-automl 
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.


Deployment failed:  (DeploymentFailed) At least one resource deployment operation failed. Please list deployment operations for details. Please see https://aka.ms/DeployOperations for usage details.


Traceback (most recent call last):
  File "/home/schrodinger/anaconda3/envs/dpv2sdk/lib/python3.7/site-packages/azure/core/polling/base_polling.py", line 482, in run
    self._poll()
  File "/home/schrodinger/anaconda3/envs/dpv2sdk/lib/python3.7/site-packages/azure/core/polling/base_polling.py", line 521, in _poll
    raise OperationFailed("Operation failed or canceled")
azure.core.polling.base_polling.OperationFailed: Operation failed or canceled

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "<ipython-input-17-64d414b6f02d>", line 54, in <module>
    client.endpoints.create(online_endpoint)
  File "/home/schrodinger/automl/sdk-cli-v2/src/azure-ml/azure/ml/_operations/endpoint_operations.py", line 227, in create
    return self._create_online_endpoint(internal_endpoint=endpoint, no_wait=no_wait)
  File "/home/schrodinger/automl/sdk-cli-v2/src/azure-ml/azure/ml/_operations/endpoint_operations.py", line 615, in _create_onl